In [47]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


def parse_trades(url):
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"Failed to load page {url}")
    
    soup = BeautifulSoup(response.content, 'html.parser')
    trades = []
    
    # Assuming trades are in a table, you might need to adjust the selectors based on the actual HTML structure
    table = soup.find('table')
    if not table:
        raise Exception("No table found on the page")
    
    headers = [header.text for header in table.find_all('th')]
    rows = table.find_all('tr')[1:]  # Skip the header row

    for row in rows:
        cells = row.find_all('td')
        if len(cells) == len(headers):  # Ensure the row has the correct number of cells
            trade = {headers[i]: cells[i].text.strip() for i in range(len(cells))}
            trades.append(trade)
    
    print(f"Number of rows parsed: {len(trades)}")  # Debugging statement
    
    return trades


url = "https://www.capitoltrades.com/trades?politician=P000197&pageSize=96"
signals = parse_trades(url)
cols = ['Traded Issuer', 'Published', 'Type']
signals = pd.DataFrame(signals)[cols].rename(columns={'Traded Issuer': 'Company', 'Published': 'Date', 'Type': 'buy_sell'})
signals = signals.query('buy_sell == "buy" or buy_sell == "sell"')
company_tickers = {
    'Alphabet IncGOOGL:US': 'GOOGL',
    'Amazon.com IncAMZN:US': 'AMZN',
    'Apple IncAAPL:US': 'AAPL',
    'NVIDIA CorporationNVDA:US': 'NVDA',
    'Palo Alto Networks IncPANW:US': 'PANW',
    'TEMPUS AI INCTEM:US': 'TEM',
    'Vistra CorpVST:US': 'VST',
    'REOF XXVI LLCN/A': 'N/A',
    'Microsoft CorpMSFT:US': 'MSFT',
    'Broadcom IncAVGO:US': 'AVGO',
    'Tesla IncTSLA:US': 'TSLA',
    'Visa IncV:US': 'V',
    'FORGE INVESTMENTS LLCN/A': 'N/A',
    'REOF XXV LLCN/A': 'N/A',
    'Roblox CorpRBLX:US': 'RBLX',
    'AllianceBernstein Holding LPAB:US': 'AB',
    'Netflix IncNFLX:US': 'NFLX',
    'PayPal Holdings IncPYPL:US': 'PYPL',
    'REOF XX LLCN/A': 'N/A',
    'salesforce.com IncCRM:US': 'CRM',
    'The Walt Disney CoDIS:US': 'DIS',
    'Micron Technology IncMU:US': 'MU',
    'Warner Bros Discover IncWBD:US': 'WBD',
    'AT&T IncT:US': 'T'
}
signals['Company'] = signals['Company'].map(company_tickers)

from datetime import datetime

signals['Date'].unique()
date_mapping = {
    '20 Jan2025': datetime(2025, 1, 20),
    '12 Sept2024': datetime(2024, 9, 12),
    '31 Jul2024': datetime(2024, 7, 31),
    '3 Jul2024': datetime(2024, 7, 3),
    '22 Mar2024': datetime(2024, 3, 22),
    '26 Feb2024': datetime(2024, 2, 26),
    '22 Dec2023': datetime(2023, 12, 22),
    '23 Jun2023': datetime(2023, 6, 23),
    '7 Apr2023': datetime(2023, 4, 7),
    '26 Jan2023': datetime(2023, 1, 26),
    '13 Jan2023': datetime(2023, 1, 13),
    '1 Dec2022': datetime(2022, 12, 1),
    '17 Oct2022': datetime(2022, 10, 17),
    '9 Sept2022': datetime(2022, 9, 9),
    '27 Jul2022': datetime(2022, 7, 27),
    '14 Jul2022': datetime(2022, 7, 14),
    '6 Jun2022': datetime(2022, 6, 6),
    '22 Mar2022': datetime(2022, 3, 22)
}
signals['Date'] = signals['Date'].map(date_mapping)
signals['Date'] = signals['Date'].dt.strftime('%Y-%m-%d')
signals.head()

Number of rows parsed: 56


,Company,Date,buy_sell
0,GOOGL,2025-01-20,buy
1,AMZN,2025-01-20,buy
2,AAPL,2025-01-20,sell
3,NVDA,2025-01-20,sell
4,NVDA,2025-01-20,buy


In [48]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

# List of companies
companies = signals['Company'].unique()

# Define the time period
end_date = datetime.now()
start_date = end_date - timedelta(days=3*365)

# Fetch data for each company
all_data = pd.DataFrame()

for company in companies:
    ticker = yf.Ticker(company)
    data = ticker.history(start=start_date, end=end_date).reset_index()
    data['Company'] = company  # Add a column for the company name
    all_data = pd.concat([all_data, data])

print("Data collection complete.")
all_data['Date'] = all_data['Date'].dt.tz_convert(None)
all_data['Date'] = all_data['Date'].dt.strftime('%Y-%m-%d')
all_data.head()

Failed to get ticker 'N/A' reason: Expecting value: line 1 column 1 (char 0)
$N/A: possibly delisted; no timezone found
/var/folders/1m/twlzmm950ps5wcz0nsl3syhr0000gn/T/ipykernel_80950/1364498156.py:19: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_data = pd.concat([all_data, data])


Data collection complete.


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Company,Adj Close
0,2022-02-22,129.369022,131.703059,127.760351,129.344620,44990000.0,0.0,0.0,GOOGL,NaN
1,2022-02-23,131.172986,131.574029,127.066869,127.127151,32786000.0,0.0,0.0,GOOGL,NaN
2,2022-02-24,124.502154,132.519590,124.502154,132.211700,54456000.0,0.0,0.0,GOOGL,NaN
3,2022-02-25,133.069608,134.782895,131.275605,133.973831,36406000.0,0.0,0.0,GOOGL,NaN
4,2022-02-28,132.569429,135.261163,132.113577,134.569183,38864000.0,0.0,0.0,GOOGL,NaN


In [57]:
import matplotlib.pyplot as plt

# Filter data for NVIDIA
nvidia_data = all_data[all_data['Company'] == 'NVDA']

# Calculate cumulative return
nvidia_data['Cumulative Return'] = (nvidia_data['Close'] / nvidia_data['Close'].iloc[0]) - 1

# Merge signals with NVIDIA data
nvidia_signals = signals[signals['Company'] == 'NVDA']
nvidia_signals.drop(columns=['Company'], inplace=True)
nvidia_data = nvidia_data.join(nvidia_signals.set_index('Date'), on=['Date'], how='left')
nvidia_data

/var/folders/1m/twlzmm950ps5wcz0nsl3syhr0000gn/T/ipykernel_80950/1696258428.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nvidia_data['Cumulative Return'] = (nvidia_data['Close'] / nvidia_data['Close'].iloc[0]) - 1
/var/folders/1m/twlzmm950ps5wcz0nsl3syhr0000gn/T/ipykernel_80950/1696258428.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nvidia_signals.drop(columns=['Company'], inplace=True)


,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Company,Adj Close,Cumulative Return,buy_sell
0,2022-02-22,22.996748,24.024039,22.961806,23.351158,633422000.0,0.0,0.0,NVDA,NaN,0.000000,NaN
1,2022-02-23,23.762482,24.114897,22.263975,22.349831,566511000.0,0.0,0.0,NVDA,NaN,-0.042881,NaN
2,2022-02-24,20.980105,23.760480,20.855312,23.708567,735801000.0,0.0,0.0,NVDA,NaN,0.015306,NaN
3,2022-02-25,23.681615,24.176790,23.342179,24.116890,528866000.0,0.0,0.0,NVDA,NaN,0.032792,NaN
4,2022-02-28,23.952166,24.624049,23.667639,24.344513,477319000.0,0.0,0.0,NVDA,NaN,0.042540,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
745,2025-02-11,132.580002,134.479996,131.020004,132.800003,178902400.0,0.0,0.0,NVDA,NaN,4.687084,NaN
746,2025-02-12,130.020004,132.240005,129.080002,131.139999,160278600.0,0.0,0.0,NVDA,NaN,4.615996,NaN
747,2025-02-13,131.559998,136.500000,131.169998,135.289993,197430000.0,0.0,0.0,NVDA,NaN,4.793717,NaN
748,2025-02-14,136.479996,139.250000,135.500000,138.850006,194892300.0,0.0,0.0,NVDA,NaN,4.946172,NaN


In [58]:
nvidia_data.query('buy_sell.notna()')

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Company,Adj Close,Cumulative Return,buy_sell
98,2022-07-14,15.088675,15.453208,14.709161,15.352337,456235000.0,0.0,0.0,NVDA,NaN,-0.342545,buy
107,2022-07-27,17.010213,17.913057,16.891365,17.767244,569776000.0,0.0,0.0,NVDA,NaN,-0.239128,sell
164,2022-10-17,11.506684,11.936262,11.505686,11.876321,580774000.0,0.0,0.0,NVDA,NaN,-0.491403,sell
462,2023-12-22,49.181057,49.369003,48.453262,48.816162,252507000.0,0.0,0.0,NVDA,NaN,1.090524,buy
593,2024-07-03,121.641220,128.260193,121.341263,128.260193,215749000.0,0.0,0.0,NVDA,NaN,4.492669,buy
612,2024-07-31,112.882564,118.321718,110.862872,117.001923,473174200.0,0.0,0.0,NVDA,NaN,4.010540,buy


In [ ]:

# Calculate strategy returns
action_signals = {'buy': 1, 'sell': -1}
nvidia_signals['Signal'] = nvidia_signals['buy_sell'].map(action_signals)
nvidia_signals['Strategy Return'] = nvidia_signals['Signal'].shift(1) * nvidia_signals['Close'].pct_change()

# Calculate cumulative strategy return
nvidia_signals['Cumulative Strategy Return'] = (1 + nvidia_signals['Strategy Return'].fillna(0)).cumprod() - 1

# Plot cumulative strategy return
plt.figure(figsize=(10, 6))
plt.plot(nvidia_signals['Date'], nvidia_signals['Cumulative Strategy Return'], label='Strategy')
plt.xlabel('Date')
plt.ylabel('Cumulative Return')
plt.title('NVIDIA Strategy Cumulative Return')
plt.legend()
# plt.grid(True)
plt.show()